<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2023/blob/main/notebooks/Topic_Modeling_%D0%B2_%D0%9F%D0%B8%D1%82%D0%BE%D0%BD%D0%B5_DH_%D0%BC%D0%B0%D0%B3%D0%B8%D1%81%D1%82%D1%80%D1%8B_2022_04_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тематическое моделирование

(напоминаю, что есть [мое видео с рассказом о ТМ](https://youtu.be/aWb8ETiZjvI))

Задача тематического моделирования состоит в том, чтобы разложить слова, употребляющиеся в корпусе на "темы" (тематически связанные мешки слов) и приписать эти темы каждому документу в корпусе. Простой пример. Есть 5 документов с такими текстами:

* Document 1: I had a peanut butter sandwich for breakfast.
* Document 2: I like to eat almonds, peanuts and walnuts.
* Document 3: My neighbor got a little dog yesterday.
* Document 4: Cats and dogs are mortal enemies.
* Document 5: You mustn’t feed peanuts to your dog.

Некоторая модель тематического моделирования может выделить такие топики (число перед словом -- некоторый параметр значимости слова для этого топика):

* Topic 1: 30% peanuts, 15% almonds, 10% breakfast…
* Topic 2: 20% dogs, 10% cats, 5% peanuts…

Дальше мы можем получить распределение по документам:

* Documents 1 and 2: 100% Topic 1
* Documents 3 and 4: 100% Topic 2
* Document 5: 70% Topic 1, 30% Topic 2

## Зачем?

Во-первых, как способ Distant Reading. Вот [мой личный опыт](https://knife.media/knife-data/)  (и пошаговое видео как я это делал - [раз](https://www.youtube.com/watch?v=yuEfqgNIz9E&t=1500s), [два](https://www.youtube.com/watch?v=NXI0_YUAtow) — правда, там не на питоне)

Помимо анализа тематического наполнения корпуса, тематическое моделирование может использоваться для:

1) построения вектроных представлений текстов (числа показывают насколько тема выражена в тексте). Такие представления могут быть лучше обычных (TfidfVectorizer, CountVectorizer), т.к. позволяют находить близкие документы даже если в них нет одинаковых слов (но есть слова одной тематики). Тематические векторы могут использоваться для поиска, рекомендаций или как признаки в классификации.

2) приписывания тэгов текстам. Так можно автоматически выделять в текстовых потоках тренды, горячие темы. Тэги можно использовать как фильтр в поисковых системах. Тэги нужно приписывать темам вручную, но это проще чем размечать обучающую выборку, так как слова в тематике сразу подказывают название.

3) составления тематических словарей

Кроме обычных текстов тематическое моделирование применяется к многим другим данным: запросам пользователей, сайтам, на которые заходят пользователи, музыке, которую слушают пользователи, покупкам в одной корзине, банковским транзакциям и даже днк.

## Зачем это нужно в DH:

* Вот тут я рассказываю про литературный кейс: https://youtu.be/JpKLjiFhXYw
* Вот тут про исторический: https://youtu.be/jgNYkeDJ45o


## Как это работает?

Все подходы к тематическому моделированию так или иначе основнованы на:

1. модели мешка слов (т.е. порядок слов в документах не учитывается)
2. независимости документов между собой (т.е. употребление слова W в тексте D_1 никак не влияет на слова в документе D_2)
3. дистрибутивной гипотезе (слова, употребляющиеся вместе, объединяются в темы)

В этой тетрадке для получения тематических моделей используются LDA из gensim. LDA -- один из самых популярных алгоритмов тематического моделирования. Вот одно из сравнительно доступных описаний его работы:
`
* *Go through each document and randomly assign each word in the document to one of K topics (K is chosen beforehand)*
* *This random assignment gives topic representations of all documents and word distributions of all the topics, albeit not very good ones*
* *So, to improve upon them:*
    * *For each document d, go through each word w and compute:*
        * *p(topic t | document d): proportion of words in document d that are assigned to topic t*
        * *p(word w| topic t): proportion of assignments to topic t, over all documents d, that come from word w*
    * *Reassign word w a new topic t’, where we choose topic t’ with probability p(topic t’ | document d) * p(word w | topic t’) This generative model predicts the probability that topic t’ generated word w*

* *On repeating the last step a large number of times, we reach a steady state where topic assignments are pretty good. These assignments are then used to determine the topic mixtures of each document.*

В питоне реализация LDA есть в популярной NLP-библиотеке gensim

Разумеется, LDA реализован не только в Python -- например, многие цифровые гуманитарии предпочитают утилитку MALLET, которая запускается в командной строке. Когда-то я написал довольно подробный [туториал](https://sysblok.ru/nlp/ishhem-smysly-kak-sdelat-tematicheskoe-modelirovanie-korpusa-tekstov/) по тому, как делать ТМ в MALLET.

Про LDA (и в целом тематическое моделирование) можно почитать вот [эту статью](https://sysblok.ru/knowhow/kak-ponjat-o-chem-tekst-ne-chitaja-ego/)

## Пробуем алгоритм на основе LDA (Latent Dirichlet Allocation) из библиотеки Gensim

Для начала попробуем на том же корпусе, на котором тестировали ключевые слова.

### Подготовка

Подготовительные мероприятия — такие же, как во многих других ситуациях: собираем корпус, предобрабатываем, лемматизируем

#### Сбор


In [ ]:
!wget https://github.com/dhhse/dh2020/raw/master/topic_modeling/LEMMATIZED_KNIFE.zip

--2022-04-21 17:01:05--  https://github.com/dhhse/dh2020/raw/master/topic_modeling/LEMMATIZED_KNIFE.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhhse/dh2020/master/topic_modeling/LEMMATIZED_KNIFE.zip [following]
--2022-04-21 17:01:05--  https://raw.githubusercontent.com/dhhse/dh2020/master/topic_modeling/LEMMATIZED_KNIFE.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16367877 (16M) [application/zip]
Saving to: ‘LEMMATIZED_KNIFE.zip.1’

LEMMATIZED_KNIFE.zi 100%[===================>]  15.61M  --.-KB/s    in 0.1s    

2022-04-21 17:01:05 (138 MB/s) - ‘LEMMATIZED_KNIFE.zip.1’ saved [163

In [ ]:
!unzip LEMMATIZED_KNIFE.zip

Сложим все тексты в один список с текстами

In [ ]:
import os

In [ ]:
full_texts = []
folder_name = 'LEMMATIZED_KNIFE'
for filename in os.listdir(folder_name):
    with open(os.path.join(folder_name, filename)) as open_file:
        text = open_file.read()
        full_texts.append(text)

In [ ]:
print(f'В корпусе {len(full_texts)} текстов')

В корпусе 6756 текстов


### Предобработка: токенизация, стоп-слова, лемматизация (в нашем случае уже сделана заранее)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import download as nltk_download
nltk_download ('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
with open ('stop_ru.txt', 'r') as stop_file:
    rus_stops = [word.strip() for word in stop_file.readlines()]

--2022-04-21 17:05:20--  https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5823 (5.7K) [text/plain]
Saving to: ‘stop_ru.txt.1’

stop_ru.txt.1       100%[===================>]   5.69K  --.-KB/s    in 0s      

2022-04-21 17:05:20 (54.6 MB/s) - ‘stop_ru.txt.1’ saved [5823/5823]



In [ ]:
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'

In [ ]:
filter = rus_stops + list(punctuation)

In [ ]:
def preprocess(input_text):
    '''функция для предобработки текста'''
    ## токенизируем через nltk:
    tokenized_text = word_tokenize(input_text.lower())
    ## убираем пунктуацию и стоп-слова:
    output_text = [word for word in tokenized_text if word not in filter]
    clean_text = [word for word in output_text if word.isalpha()]
    return clean_text

In [ ]:
# предобработка занимает время, поэтому включим показ прогресс-бара
from tqdm import tqdm

In [ ]:
preprocessed_texts = [preprocess(text) for text in tqdm(full_texts)]

NameError: ignored

In [ ]:
preprocessed_texts

NameError: ignored

Посмотрим, как оно выглядит в предобработанном виде:

In [ ]:
preprocessed_texts[0:3]

NameError: ignored

### Теперь можно делать собственно топик моделинг:

In [ ]:
import gensim

Список токенизированных и лемматизированных файлов надо превратить в специальный объект `gensim.corpora.Dictionary`:

In [ ]:
gensim_dictionary_for_TM = gensim.corpora.Dictionary(preprocessed_texts)

Метод .filter_extremes удаляет из нашего корпуса слова, которые
* встречаются **более** чем в каком-то **проценте** от документов нашего корпуса (параметр `no_above`, ниже он стоит 10%)
* встречаются **менее** чем в каком-то **числе** документов корпуса
(параметр `no_below`, ниже он стоит 20 документов):

In [ ]:
gensim_dictionary_for_TM.filter_extremes(no_above=0.1, no_below=20)

После `filter_extremes` делают `.compactify()`, чтобы немножко сжать ваш генсим-словарь: в нем удалено много слов и освободились какие-то id. Значит, можно перераспредилить id, чтобы в среднем они стали покороче и занимали меньше места:

In [ ]:
gensim_dictionary_for_TM.compactify()

In [ ]:
print(gensim_dictionary_for_TM)

Dictionary(10120 unique tokens: ['алкоголь', 'безопасный', 'вечер', 'влиять', 'вывод']...)


Преобразуем наши тексты в мешки слов с помощью встроенного в генсимовский словарь метода `.doc2bow` (т.е. document to bag-of-words)

In [ ]:
corpus = [gensim_dictionary_for_TM.doc2bow(text) for text in preprocessed_texts]

Все, теперь можно делать топик моделинг. Для LDA применяется `gensim.models.LdaMulticore`. Обучние модели на этом корпусе занимает около минуты

In [ ]:
lda = gensim.models.LdaMulticore(corpus,
                                 num_topics = 20, # число топиков
                                 id2word=gensim_dictionary_for_TM,
                                 passes=10)

In [ ]:
lda.print_topics()

[(0,
  '0.018*"мозг" + 0.008*"сон" + 0.006*"организм" + 0.006*"пациент" + 0.005*"врач" + 0.005*"клетка" + 0.005*"вещество" + 0.005*"расстройство" + 0.005*"болезнь" + 0.004*"заболевание"'),
 (1,
  '0.018*"отходы" + 0.011*"мусор" + 0.009*"переработка" + 0.009*"бутылка" + 0.008*"сингл" + 0.008*"альбом" + 0.008*"сбор" + 0.007*"пластик" + 0.007*"пластиковый" + 0.007*"раздельный"'),
 (2,
  '0.013*"технология" + 0.011*"искусственный" + 0.010*"интеллект" + 0.010*"задача" + 0.008*"обучение" + 0.008*"робот" + 0.007*"алгоритм" + 0.007*"студент" + 0.006*"нейросеть" + 0.006*"олимпиада"'),
 (3,
  '0.009*"теория" + 0.008*"философ" + 0.007*"философия" + 0.006*"право" + 0.004*"человеческий" + 0.004*"мысль" + 0.004*"прошлое" + 0.004*"природа" + 0.004*"квантовый" + 0.003*"иной"'),
 (4,
  '0.018*"пользователь" + 0.010*"соцсеть" + 0.009*"сеть" + 0.009*"интернет" + 0.008*"приложение" + 0.006*"фейсбук" + 0.006*"доллар" + 0.005*"команда" + 0.005*"контент" + 0.005*"доступ"'),
 (5,
  '0.007*"государство" + 0.00

Посмотреть распределение на конкретном тексте:

In [ ]:
lda[corpus[0]]

[(0, 0.84305114), (4, 0.1411594)]

In [ ]:
full_texts[0]

'голландский ученый заявлять, что псилоцибиновый (галлюциногенный) гриб положительно влиять на креативность и эмпатия человек. эффект сохраняться в течение неделя после употребление, говорить исследователь. в эксперимент принимать участие 55 доброволец, половина из который рано уже пробовать псилоцибин. испытуемый пить специальный грибной отвар, а затем проходить творческий и психологический тест. доброволец тестировать по три раз: вечер перед прием отвар, сразу после прием и спустя неделя. ученый оценивать креативность, эмпатия и общий уровень тревожность человек. исследователь приходить к вывод, что отвар улучшать самочувствие группа и делать человек более открытый. «этот результат очень важный для понимание терапевтический ценность псилоцибин при лечение тревожность, депрессия и посттравматический стрессовый расстройство», — говорить наташа мейсон, один из автор исследование. в май прошлый год псилоцибиновый гриб признавать в пять раз безопасный, чем \xa0кокаин, ЛСД и мдма. опасност

In [ ]:
os.listdir(folder_name)[0]

'magic-mushrooms.txt'

In [ ]:
lda[corpus[2372]]

[(4, 0.13739039),
 (5, 0.22777633),
 (7, 0.49837863),
 (13, 0.055496234),
 (18, 0.06706948)]

In [ ]:
os.listdir(folder_name)[2372]

'nelyubov-oscar.txt'

In [ ]:
full_texts[2372]

'фильм «нелюбовь» российский режиссер андрей звягинцев номинировать на премия «оскар» в категория «хороший фильм на иностранный язык», сообщать «интерфакс». кроме лента звягинцев, за награда быть бороться «фантастический женщина» (чили), «оскорбление» (ливан), «квадрат» (швеция) и «о тело и душа» (венгрия). «нелюбовь» удостаиваться приз жюри каннский фестиваль, а также\xa0завоевывать\xa0гран-при неделя российский кино в лондон. фильм рассказывать о семья, переживать тяжелый развод. за приз хороший режиссер побороться грета гервиг («леди бердо»), гильермо дель торо («форма вода»), кристофер нолан («дюнкерк»), джордан пить («прочь»), пол томас андерсон («призрачный нить»).\xa090-я церемония вручение награда американский киноакадемия пройти 4 март в лос-анджелес. \n'

Основные параметры, которыми мы можем управлять, это num_topics и passes.

**num_topics** - это количество тем. Его обычно подбирают либо на глаз, глядя на то, насколько хорошо интерпретируемыми получаются темы, либо с использованием специальных метрик типа 'перплексии', которые оценивают однородность топиков автоматически. По умолчанию в генсиме он 100.  

**passes** - задает количество проходов по данным. Чем больше, тем лучше сойдется модель, но обучаться будет дольше.

Про параметры можно почитать в документации:

In [ ]:
?gensim.models.LdaMulticore

Визуализация:

Можно сделать с помощью pyLDAvis — специальной библиотеки для визуализации результатов LDA в питоне.

In [ ]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f55a1e50acd9fa963eb2c7f1a2d0cfccb6c096917164d2a7da6b2e23c666ae79
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
vis = gensimvis.prepare(lda, corpus, gensim_dictionary_for_TM)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.076228 -0.127879       1        1  11.021146
0     -0.207919  0.033695       2        1   9.388834
14    -0.149568 -0.010437       3        1   9.235062
12     0.105259  0.039613       4        1   8.234034
6      0.014260  0.079566       5        1   6.547280
3     -0.113028 -0.100590       6        1   6.447393
15     0.039275 -0.128866       7        1   5.228656
4      0.021291  0.105204       8        1   4.684469
16    -0.065667 -0.143640       9        1   4.616962
5      0.020835 -0.033156      10        1   4.533008
9     -0.034843 -0.124801      11        1   4.518286
7      0.129192 -0.052337      12        1   4.086790
11     0.112385 -0.042547      13        1   3.522305
17    -0.092685  0.048493      14        1   3.486542
8     -0.069852  0.203065      15        1   3.415897
18     0.253249  0.065818      16        1   3.113986
2     -0.071528  0.081965      17        1   3.086924
10     0.099836 -0.081547      18        1   2.978098
13     0.100874 -0.007205      19        1   1.110207
1     -0.015137  0.195588      20        1   0.744123, topic_info=              Term         Freq        Total Category  logprob  loglift
712           мозг  2475.000000  2475.000000  Default  30.0000  30.0000
1192     искусство  1822.000000  1822.000000  Default  29.0000  29.0000
586         музыка  1634.000000  1634.000000  Default  28.0000  28.0000
1781  пользователь  1166.000000  1166.000000  Default  27.0000  27.0000
1378      художник  1144.000000  1144.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
1636  производство    57.523214   690.321203  Topic20  -5.1540   2.4158
1017      материал    54.717912   750.888483  Topic20  -5.2040   2.2817
2743             s    45.253460   268.029309  Topic20  -5.3939   3.1219
797    производить    46.573257   557.846052  Topic20  -5.3651   2.4177
40            тест    41.153082   670.171229  Topic20  -5.4889   2.1105

[1324 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
91         1  0.066659            a
91         2  0.011493            a
91         4  0.009194            a
91         6  0.039076            a
91         8  0.002299            a
...      ...       ...          ...
6926       2  0.970073   яйцеклетка
10119      5  0.181361  ярославский
10119      6  0.103635  ярославский
10119     16  0.647717  ярославский
10119     17  0.077726  ярославский

[8941 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 1, 15, 13, 7, 4, 16, 5, 17, 6, 10, 8, 12, 18, 9, 19, 3, 11, 14, 2])

In [ ]:
lda.print_topics()

[(0,
  '0.018*"мозг" + 0.008*"сон" + 0.006*"организм" + 0.006*"пациент" + 0.005*"врач" + 0.005*"клетка" + 0.005*"вещество" + 0.005*"расстройство" + 0.005*"болезнь" + 0.004*"заболевание"'),
 (1,
  '0.018*"отходы" + 0.011*"мусор" + 0.009*"переработка" + 0.009*"бутылка" + 0.008*"сингл" + 0.008*"альбом" + 0.008*"сбор" + 0.007*"пластик" + 0.007*"пластиковый" + 0.007*"раздельный"'),
 (2,
  '0.013*"технология" + 0.011*"искусственный" + 0.010*"интеллект" + 0.010*"задача" + 0.008*"обучение" + 0.008*"робот" + 0.007*"алгоритм" + 0.007*"студент" + 0.006*"нейросеть" + 0.006*"олимпиада"'),
 (3,
  '0.009*"теория" + 0.008*"философ" + 0.007*"философия" + 0.006*"право" + 0.004*"человеческий" + 0.004*"мысль" + 0.004*"прошлое" + 0.004*"природа" + 0.004*"квантовый" + 0.003*"иной"'),
 (4,
  '0.018*"пользователь" + 0.010*"соцсеть" + 0.009*"сеть" + 0.009*"интернет" + 0.008*"приложение" + 0.006*"фейсбук" + 0.006*"доллар" + 0.005*"команда" + 0.005*"контент" + 0.005*"доступ"'),
 (5,
  '0.007*"государство" + 0.00